<a href="https://colab.research.google.com/github/tjunjet/til_nothing/blob/YF_NLP/NLP_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference made to: https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5

Preprocessing for NLP- input: wav files, output: dataloader
Edits are explained in the comments

Plug n Play instructions (using GCOllab)
- create a directory called data under the current GCollab (content) directory
- create additional subdirectories for each sentiment and throw the wav files in the relevant directories 

In [173]:
import os
import pandas as pd

In [174]:
directory= "/content/data" #change this to whatever directory you need- take note it can be relative or absolute (there will be a base address declared subsequently)
classID={}
for idx, directory in enumerate (os.listdir(directory)):
  classID[idx]= directory

In [175]:
fileID={}
for id,item in classID.items():
  directory_path= "/content/data/{}".format(item)
  for file in enumerate (os.listdir(directory_path)):
    file_path= directory_path+ "/" + file[1]
    fileID[file_path]= int(id)

In [176]:
df= pd.DataFrame.from_dict(fileID,orient='index')
df= df.reset_index()
df.columns= ["relative_path", "classID"]
df

,relative_path,classID
0,/content/data/happy/2aae288dc3.wav,0
1,/content/data/happy/0cb2b21058.wav,0
2,/content/data/happy/1d09cd2060.wav,0
3,/content/data/happy/2ab7c404aa.wav,0
4,/content/data/happy/1d75944045.wav,0
...,...,...
65,/content/data/sad/0000d71f36.wav,1
66,/content/data/sad/00f39c35c4.wav,1
67,/content/data/sad/0a12de9ea3.wav,1
68,/content/data/sad/0fcca4a43b.wav,1


In [178]:
#for some reason i cant get rid of the ipynb checkpoints when uploading onto GoogleCollab- either remove this if you are not using GCOllab or add for each catgeory (ie angry neutral etc)
#Only experimented with 2 emotions
df.drop(df[(df["relative_path"]=="/content/data/happy/.ipynb_checkpoints")].index,inplace=True)
df.drop(df[(df["relative_path"]=="/content/data/sad/.ipynb_checkpoints")].index,inplace=True)
df= df.reset_index()
df
#stick to the same format for dataloader to load data properly

,level_0,index,relative_path,classID
0,0,0,/content/data/happy/2aae288dc3.wav,0
1,1,1,/content/data/happy/0cb2b21058.wav,0
2,2,2,/content/data/happy/1d09cd2060.wav,0
3,3,3,/content/data/happy/2ab7c404aa.wav,0
4,4,4,/content/data/happy/1d75944045.wav,0
...,...,...,...,...
63,63,65,/content/data/sad/0000d71f36.wav,1
64,64,66,/content/data/sad/00f39c35c4.wav,1
65,65,67,/content/data/sad/0a12de9ea3.wav,1
66,66,68,/content/data/sad/0fcca4a43b.wav,1


In [152]:
#Copy pasta :D
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio

class AudioUtil():
  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------
  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)
  # ----------------------------
  # Convert the given audio to the desired number of channels
  # ----------------------------
  @staticmethod
  def rechannel(aud, new_channel):
    sig, sr = aud

    if (sig.shape[0] == new_channel):
      # Nothing to do
      return aud

    if (new_channel == 1):
      # Convert from stereo to mono by selecting only the first channel
      resig = sig[:1, :]
    else:
      # Convert from mono to stereo by duplicating the first channel
      resig = torch.cat([sig, sig])
    return ((resig, sr))
  # ----------------------------
  # Since Resample applies to a single channel, we resample one channel at a time
  # ----------------------------
  @staticmethod
  def resample(aud, newsr):
    sig, sr = aud

    if (sr == newsr):
      # Nothing to do
      return aud

    num_channels = sig.shape[0]
    # Resample first channel
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      # Resample the second channel and merge both channels
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  #   ----------------------------
  # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds
  # ----------------------------
  @staticmethod
  def pad_trunc(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms

    if (sig_len > max_len):
      # Truncate the signal to the given length
      sig = sig[:,:max_len]

    elif (sig_len < max_len):
      # Length of padding to add at the beginning and end of the signal
      pad_begin_len = random.randint(0, max_len - sig_len)
      pad_end_len = max_len - sig_len - pad_begin_len

      # Pad with 0s
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))
    return (sig, sr)
  # ----------------------------
  # Shifts the signal to the left or right by some percent. Values at the end
  # are 'wrapped around' to the start of the transformed signal.
  # ----------------------------
  @staticmethod
  def time_shift(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)
  # ----------------------------
  # Generate a Spectrogram
  # ----------------------------
  @staticmethod
  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80

    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

    # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)
  # ----------------------------
  # Augment the Spectrogram by masking out some sections of it in both the frequency
  # dimension (ie. horizontal bars) and the time dimension (vertical bars) to prevent
  # overfitting and to help the model generalise better. The masked sections are
  # replaced with the mean value.
  # ----------------------------
  @staticmethod
  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec

In [179]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio

# ----------------------------
# Sound Dataset
# ----------------------------
class SoundDS(Dataset):
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 1000 #initial 4000 value was giving issues- suspect it is due to how short our datasets are- maybe can increase to 1500
    self.sr = 44100
    self.channel = 2
    self.shift_pct = 0.4
            
  # ----------------------------
  # Number of items in dataset
  # ----------------------------
  def __len__(self):
    return len(self.df)    
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
  def __getitem__(self, idx):
    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    # Get the Class ID
    class_id = self.df.loc[idx, 'classID']

    aud = AudioUtil.open(audio_file)
    # Some sounds have a higher sample rate, or fewer channels compared to the
    # majority. So make all sounds have the same number of channels and same 
    # sample rate. Unless the sample rate is the same, the pad_trunc will still
    # result in arrays of different lengths, even though the sound duration is
    # the same.
    reaud = AudioUtil.resample(aud, self.sr)
    rechan = AudioUtil.rechannel(reaud, self.channel)
    dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
    shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
    sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

    return aug_sgram, class_id

In [169]:
from torch.utils.data import random_split

data_path= "" #this is the base address if you used relative path above (i used absolute path initially so its empty)
myds = SoundDS(df, data_path)

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(myds, batch_size=16, shuffle=True) #i just loaded the whole data set in for now

In [165]:
dir(train_dl) #cool way to see attributes of an object

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_index_sampler',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_init_fn']

In [180]:
it=iter(train_dl)
aug_sgram, class_id=it.next()
aug_sgram

tensor([[[[ 1.3573e+01, -7.3467e+00,  8.5993e-01,  ..., -1.0923e+01,
           -4.8018e+00, -3.3703e+00],
          [ 1.4747e+01, -6.6490e+00, -5.0097e+00,  ..., -1.0408e+01,
            2.2515e+00, -2.0322e+00],
          [ 2.0546e+01, -1.9601e+00,  4.1181e+00,  ..., -1.7096e+01,
            6.1800e+00,  1.2523e+01],
          ...,
          [-2.8278e+01, -4.3915e+01, -4.3915e+01,  ..., -4.3915e+01,
           -4.3915e+01, -4.3915e+01],
          [-2.8284e+01, -4.3915e+01, -4.3915e+01,  ..., -4.3915e+01,
           -4.3915e+01, -4.3915e+01],
          [-2.8209e+01, -4.3915e+01, -4.3915e+01,  ..., -4.3915e+01,
           -4.3915e+01, -4.3915e+01]],

         [[ 1.3573e+01, -7.3467e+00,  8.5993e-01,  ..., -1.0923e+01,
           -4.8018e+00, -3.3703e+00],
          [ 1.4747e+01, -6.6490e+00, -5.0097e+00,  ..., -1.0408e+01,
            2.2515e+00, -2.0322e+00],
          [ 2.0546e+01, -1.9601e+00,  4.1181e+00,  ..., -1.7096e+01,
            6.1800e+00,  1.2523e+01],
          ...,
     

In [181]:
class_id

tensor([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1])